In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import cv2
from skimage import io

img_orig = io.imread("img/fish01.jpg")

plt.imshow(img_orig)
plt.show()

## Q.1. チャネル入れ替え
画像を読み込み、RGBをBGRに入れ替える。

In [ ]:
def rgb2bgr(img):
    return img[..., ::-1]

img_bgr = rgb2bgr(img_orig)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1); plt.title('input'); plt.imshow(img_orig)
plt.subplot(1, 2, 2); plt.title('answer'); plt.imshow(img_bgr)
plt.show()

## Q.2. グレースケール Grayscale
画像をグレースケールにする。グレースケールは以下の式で表される。
```math
Grayscale = 0.2126R + 0.7152G + 0.0722B
```

In [ ]:
img_gray = cv2.cvtColor(img_orig, cv2.COLOR_RGB2GRAY)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1); plt.title('input'); plt.imshow(img_orig)
plt.subplot(1, 2, 2); plt.title('answer'); plt.imshow(img_gray, cmap='gray')
plt.show()

## Q.3. 二値化, Binarization
画像を二値化する。二値化とは、画像を特定の値を閾値として黒と白の二値で表現する方法。
 1. グレースケール化
 2. 閾値を110にして二値化

In [ ]:
img_gray = cv2.cvtColor(img_orig, cv2.COLOR_RGB2GRAY)
th, img_bin = cv2.threshold(img_gray, 110, 255, cv2.THRESH_BINARY)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1); plt.title('input'); plt.imshow(img_orig)
plt.subplot(1, 3, 2); plt.title('gray'); plt.imshow(img_gray, cmap='gray')
plt.subplot(1, 3, 3); plt.title('answer'); plt.imshow(img_bin, cmap='gray')
plt.show()

## Q.4. 大津の二値化, Otsu's binarization
大津の二値化を実装する。
1. グレースケール化
2. 大津の二値化 \
大津の二値化とは判別分析法と呼ばれ、二値化における分離の閾値を自動決定する教師なし手法であるこれはクラス内分散とクラス間分散の比から計算される。

In [ ]:
th, img_bin = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
print('threshold >>', th)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1); plt.title('input'); plt.imshow(img_orig)
plt.subplot(1, 3, 2); plt.title('gray'); plt.imshow(img_gray, cmap='gray')
plt.subplot(1, 3, 3); plt.title('answer'); plt.imshow(img_bin, cmap='gray')
plt.show()

## Q.5. HSV変換, HSV transformation
HSV変換を実装して、色相Hを反転する。

In [ ]:
hsv = cv2.cvtColor(img_orig, cv2.COLOR_RGB2HSV) # RGB -> HSV
hsv[..., 0] = (hsv[..., 0] + 90) % 180 # Hue of opencv is defined [0, 180]
img_hsv = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB) # HSV -> RGB

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1); plt.title('input'); plt.imshow(img_orig)
plt.subplot(1, 2, 2); plt.title('answer'); plt.imshow(img_hsv)
plt.show()

### H,S,Vをそれぞれ見てみる

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 4, 1); plt.title('input'); plt.imshow(img_orig)
plt.subplot(1, 4, 2); plt.title('Hue'); plt.imshow(hsv[..., 0]/360, cmap='hsv')
plt.subplot(1, 4, 3); plt.title('Saturation'); plt.imshow(hsv[..., 1], cmap='gray')
plt.subplot(1, 4, 4); plt.title('Value'); plt.imshow(hsv[..., 2], cmap='gray')
plt.show()

### 他の画像でも見てみる

In [ ]:
img_another = io.imread('./img/fish02.jpg')

hsv = cv2.cvtColor(img_another, cv2.COLOR_RGB2HSV)

plt.figure(figsize=(12, 4))
plt.subplot(1, 4, 1); plt.title('input'); plt.imshow(img_another)
plt.subplot(1, 4, 2); plt.title('Hue'); plt.imshow(hsv[..., 0], cmap='hsv')
plt.subplot(1, 4, 3); plt.title('Saturation'); plt.imshow(hsv[..., 1], cmap='gray')
plt.subplot(1, 4, 4); plt.title('Value'); plt.imshow(hsv[..., 2], cmap='gray')
plt.show()

### Hueを見てみる

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1); plt.title('input'); plt.imshow(img_another)
plt.subplot(1, 3, 2); plt.title('input hue'); plt.imshow(hsv[..., 0]/360, cmap='hsv')
plt.subplot(1, 3, 3); plt.title('Hue color')
for i in range(360):
    plt.vlines(i, 0, 1, color=cm.hsv(i / 360))
    
plt.show()

## Q.6. 減色, color subtraction
ここでは画像の値を256^3から4^3の値に減色する。  
すなわちR,G,Bを{32, 96, 160, 224}の4値に減色する。

In [ ]:
def color_subtraction(img, div=4):
    th = 256 // div
    return np.clip(img // th * th + th // 2, 0, 255)

# 減色処理
img_sub = color_subtraction(img_orig)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1); plt.title('input'); plt.imshow(img_orig)
plt.subplot(1, 2, 2); plt.title('answer'); plt.imshow(img_sub)
plt.show()

### 減色を8値にしてみる
8値でも元の画像に相当近い見た目を保持できている。  
色空間はかなり削減できる。

In [ ]:
img_sub = color_subtraction(img_orig, div=8)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1); plt.title('input'); plt.imshow(img_orig)
plt.subplot(1, 2, 2); plt.title('answer'); plt.imshow(img_sub)
plt.show()

## Q.7. 平均プーリング, average pooling
ここでは画像をグリッド分割し、各領域内の平均値でその領域内の値を埋める。  
この操作をプーリング処理と呼ぶ。

In [ ]:
def pool_average(img, ksize_h=8, ksize_w=8):
    _img = img.copy().astype(np.float32)
    
    # padding
    h, w = img.shape[:2]
    outer_h = h % ksize_h
    pad_top = outer_h // 2
    pad_bottom = outer_h - pad_top
    outer_w = w % ksize_w
    pad_left = outer_w // 2
    pad_right = outer_w - pad_left
    
    _img = np.pad(_img, [(pad_top, pad_bottom), (pad_left, pad_right), (0, 0)], 'edge')
    out = np.zeros_like(_img)
    
    new_h, new_w = out.shape[:2]
    c = 1 if len(out.shape) == 2 else out.shape[2]
    
    # filtering
    for iy in range(0, new_h, ksize_h):
        for ix in range(0, new_w, ksize_w):
            for ic in range(c):
                out[iy:iy+ksize_h, ix:ix+ksize_w, ic] = _img[iy:iy+ksize_h, ix:ix+ksize_w, ic].mean()
                
    out = out[pad_top:pad_top+h, pad_left:pad_left+w]
    
    return np.clip(out, 0, 255).astype(np.uint8)

img_pool = pool_average(img_orig)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1); plt.title('input'); plt.imshow(img_orig)
plt.subplot(1, 2, 2); plt.title('answer'); plt.imshow(img_pool)
plt.show()

### フィルタサイズを大きくすると
フィルタサイズが大きくなると、ピンボケの度合いが強くなる

In [ ]:
img_pool = pool_average(img_orig, ksize_h=16, ksize_w=16)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1); plt.title('input'); plt.imshow(img_orig)
plt.subplot(1, 2, 2); plt.title('answer'); plt.imshow(img_pool)
plt.show()

## Q.8. 最大プーリング, max pooling
平均値ではなく、最大値でプーリング処理を行う。

In [ ]:
def pool_max(img, ksize_h=8, ksize_w=8):
    _img = img.copy().astype(np.float32)
    
    # padding
    h, w = img.shape[:2]
    outer_h = h % ksize_h
    pad_top = outer_h // 2
    pad_bottom = outer_h - pad_top
    outer_w = w % ksize_w
    pad_left = outer_w // 2
    pad_right = outer_w - pad_left
    
    _img = np.pad(_img, [(pad_top, pad_bottom), (pad_left, pad_right), (0, 0)], 'edge')
    out = np.zeros_like(_img)
    
    new_h, new_w = out.shape[:2]
    c = 1 if len(out.shape) == 2 else out.shape[2]
    
    # filtering
    for iy in range(0, new_h, ksize_h):
        for ix in range(0, new_w, ksize_w):
            for ic in range(c):
                out[iy:iy+ksize_h, ix:ix+ksize_w, ic] = _img[iy:iy+ksize_h, ix:ix+ksize_w, ic].max()
    
    out = out[pad_top:pad_top+h, pad_left:pad_left+w]
    return np.clip(out, 0, 255).astype(np.uint8)

img_pool = pool_max(img_orig)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1); plt.title('input'); plt.imshow(img_orig)
plt.subplot(1, 2, 2); plt.title('answer'); plt.imshow(img_pool)
plt.show()